In [1]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd

# Load the datasets
energy = pd.read_csv("Household_energy_data.csv")
schemes = pd.read_csv("govt schemes.csv")
alt_energy = pd.read_csv("alternate_energy_sources.csv")

# Sample data
energy = energy.sample(10000, random_state=15)
energy['Year'] = pd.to_datetime(energy['Year'], format='%Y').dt.strftime('%Y')
energy['Month'] = pd.to_datetime(energy['Month'], format='%b').dt.strftime('%b')
energy["Household_ID"] = energy["Household_ID"].astype(str)
energy_con = energy.drop(columns=['Total_Consumption_kWh'])

# Create dummy variables
data = pd.get_dummies(energy_con, columns=['Country', 'Appliance', 'Month', 'Year'])
data = data.drop(columns=['Household_ID'])

# Define features and target
Y = data['Energy_Consumption_kWh']
X = data.drop(columns=['Energy_Consumption_kWh'])

# Split data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Train the model
model = GradientBoostingRegressor(n_estimators=100, random_state=42)
model.fit(X_train, Y_train)

# Function to predict energy consumption
def predict_energy_consumption():
    global country
    try:
        while True:
            country = input("Enter country: ").strip().capitalize()
            if country in energy['Country'].unique():
                break
            else:
                print("Invalid country. Please enter a valid country from the list.")

        while True:
            month = input("Enter month (e.g., Jan, Feb, etc.): ").strip().capitalize()
            valid_months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
                            'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
            if month in valid_months:
                break
            else:
                print("Invalid month. Please enter a valid month abbreviation (e.g., Jan).")

        while True:
            year = input("Enter year (e.g., 2020): ").strip()
            if year.isdigit() and 2010 <= int(year) <= 2100:
                break
            else:
                print("Invalid year. Please enter a valid year (between 2010 and 2100).")

        while True:
            try:
                num_appliances = int(input("Enter number of appliances: "))
                if num_appliances <= 0:
                    raise ValueError("Number of appliances must be greater than zero.")
                break
            except ValueError:
                print("Invalid input. Please enter a valid number.")

        appliances = []
        for i in range(num_appliances):
            while True:
                appliance = input(f"Enter appliance {i+1}: ").strip().capitalize()
                if appliance in energy['Appliance'].unique():
                    appliances.append(appliance)
                    break
                else:
                    print("Invalid appliance. Please enter a valid appliance from the list.")

        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)

        total_prediction = 0
        for appliance in appliances:
            input_data = pd.DataFrame({
                'Country': [country],
                'Month': [month],
                'Year': [year],
                'Appliance': [appliance]
            })
            input_data = pd.get_dummies(input_data, columns=['Country', 'Appliance', 'Month', 'Year'])
            input_data = input_data.reindex(columns=X_train.columns, fill_value=0)
            input_data_scaled = scaler.transform(input_data)
            prediction = model.predict(input_data_scaled)
            total_prediction += prediction[0]

        print(f"Predicted energy consumption for {num_appliances} appliances: {round(total_prediction, 2)} kWh")
        return total_prediction

    except ValueError as ve:
        print(f"Error: {ve}. Please enter correct data.\n")
        return predict_energy_consumption()

# Predict energy consumption
predicted_energy = predict_energy_consumption()
print("The stored predicted energy value is:", predicted_energy)
print(country)

# Function to suggest alternate energy sources
def suggest_alternate_source(predicted_energy, country):
    # Filter by country and suitable energy range
    filtered_alt = alt_energy[(alt_energy['Country'] == country) &
                              (alt_energy['Min Consumption (kWh)'] <= predicted_energy) &
                              (alt_energy['Max Consumption (kWh)'] >= predicted_energy)].sort_values('Avg Installation Cost ($)')

    print("You may consider transitioning to the following alternate energy sources:")
    print(filtered_alt[['Energy Source', 'Avg Installation Cost ($)', 'Min Consumption (kWh)', 'Max Consumption (kWh)']])

    print("\nGovernment schemes available in your country:")
    relevant_schemes = schemes[schemes['Country'] == country][['Scheme/Subsidy', 'Specific Source']]
    print(relevant_schemes)

# Suggest alternate energy sources based on the predicted consumption and country
suggest_alternate_source(predicted_energy, country)


Invalid country. Please enter a valid country from the list.
Predicted energy consumption for 2 appliances: 127.88 kWh
The stored predicted energy value is: 127.88430164790068
Norway
You may consider transitioning to the following alternate energy sources:
    Energy Source  Avg Installation Cost ($)  Min Consumption (kWh)  \
386    Hydropower                     3000.0                    100   
387       Biomass                     3500.0                    100   
385    Wind Power                     5750.0                    100   
384   Solar Power                    18750.0                    100   

     Max Consumption (kWh)  
386                    200  
387                    200  
385                    200  
384                    200  

Government schemes available in your country:
                    Scheme/Subsidy Specific Source
24  Electricity Certificate Scheme     Wind, Hydro


c:\Users\apoor\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  "X does not have valid feature names, but"
c:\Users\apoor\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  "X does not have valid feature names, but"
